In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import pickle
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer
%load_ext autoreload
%autoreload 2
import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str


### CorZu 2.0

In [2]:
import os
import urllib.request
import urllib.parse

def _parse_dependencies_to_conll_file(text, conll_name):
    params = {
        "text": text,
        "format": "conll"
    }
    args = urllib.parse.urlencode(params).encode("utf-8")
    req = "http://localhost:5003/parse/?"+args.decode("utf-8")
    try:
        f = urllib.request.urlopen(req)
        if conll_name:
            file = open(conll_name,'w')
            file.write(f.read().decode('utf-8'))
        else:
            print(f.read().decode('utf-8'))
        return 0
    except Exception as ex:
        print(ex)
        return 1

def _create_markables(conll_name, markable_name):
    cmd = "python ext/CorZu_v2.0/extract_mables_from_parzu.py " + conll_name + " > " + markable_name
    err = os.system(cmd)
    return err

def execute_corzu(text, conll_name="output.conll", markable_name="markable", output="corzu_export"):
    _parse_dependencies_to_conll_file(text, conll_name)
    err = _create_markables(conll_name, markable_name)
    if err == 0:
        cmd = "python ext/CorZu_v2.0/corzu.py " + markable_name + " " + conll_name + " " + output    
        err = os.system(cmd)
        return err

0

### Read Tuebadz

In [3]:
def read_tuebadz(path):
    i = 0
    raw_data = {}
    gold_standard = {}
    sentences = []
    for line in open(path,'r').readlines():
        li=line.strip()
        if li.startswith("#"):
            li = li.split()
            if li[1] == 'newdoc':
                if 'newdoc' in raw_data:
                    sentences.append(raw_data)
                    raw_data = {}
                else:
                    raw_data['newdoc'] = li[4]
                    new_doc = li[4]
                    gold_standard[new_doc] = ""
            if li[1] == 'sent_id':
                sent_id = li[3]
                raw_data['sent_id'] = li[3]
            if li[1] == 'text':
                if "text" in raw_data:
                    raw_data["text"] += line[9:].rstrip(" \n")
                else:
                    raw_data["text"] = line[9:].rstrip(" \n") + ". "
        else:
            gold_standard[new_doc] += line
    sentences.append(raw_data)
    return sentences, gold_standard
            

raw_sentences, gold_standard = read_tuebadz("tuebadz-11.0-conll-u-v2.txt")

In [9]:
print(raw_sentences[1]["text"])
execute_corzu(raw_sentences[1]["text"])

SPD/CDU/AfB für Daewoo-Millionen. Aber Bremerhavens AfB fordert jetzt UntersuchungsausschußDie Wirtschaftsförderausschüsse haben gestern ein Finanzierungsvolumen von insgesamt 86 Millionen Mark beschlossen, mit denen das Vulkan-Gelände für die Ansiedlung des Autoimporteurs Egerland frei gemacht werden kann.Egerland soll die 10.000 bis 20.000 koreanischen Daewoo-Pkw, die derzeit von der Firma Mosolf in Bremerhaven abgeladen werden, in Vegesack abladen.Der Chef der Bremer Investitions-Gesellschaft (BIG), Ulrich Keller, erklärte dem Ausschuß, es "zeichne sich die konkrete Möglichkeit" ab, daß Daewoo auch ein Ersatzteillager in Vegesack einrichtet, berichteten SPD-Wirtschaftspolitiker.Auch die Vertreter der AfB stimmten den 86 Millionen zu, während Bremerhavens AfB-Vorsitzender Günter Diekhöner an seiner Forderung festhält, ein Untersuchungsausschuß solle die undurchsichtigen Vorgänge prüfen.Aus seiner Sicht schadet die Verlagerung des Auto-Geschäftes den Bremerhavener Interessen.Heftige K

0